In [1]:
import numpy as np
import torch
import open3d as o3d

from vehicle_reconstruction import *
from scene_segmentation import *
from open3d_vis_utils import draw_scenes

torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


device(type='cuda', index=0)

In [2]:
pts = np.load(".\\data\\00043_pts.npy").reshape(-1, 4).astype(np.float32)
gt_box = np.load(".\\data\\00043_gt.npy").reshape(-1, 7).astype(np.float32)

print(pts.shape)
print(gt_box.shape)

(162103, 4)
(72, 7)


In [3]:
vis = o3d.visualization.Visualizer()
vis.create_window(window_name="view")

draw_scenes(vis, points=pts, gt_boxes=gt_box, draw_origin=True)

vis.run()
vis.destroy_window()

In [4]:
global_res = 0.1
standard_bbox = [2.5, 1.9, 5.6, -1.25, 1.25, 0.0, 1.90, -2.8, 2.8]
grid_res = [int(np.ceil(standard_bbox[0]/global_res) + 1),
            int(np.ceil(standard_bbox[1]/global_res) + 1),
            int(np.ceil(standard_bbox[2]/global_res) + 1)]
print(grid_res)
vehi_reconstructor = vehicle_reconstructor(vehicles=None,
                                           sampling_space=standard_bbox,
                                           grid_res=grid_res,
                                           global_res=global_res)
vehi_reconstructor.load_voxel(".\\data\\preprocessed_voxel.bin")
vehi_reconstructor.fit_model(k=5)

[26, 20, 57]


In [5]:
scene = point_cloud_scene(pts=torch.from_numpy(pts).cuda(),
                          gt_boxes=torch.from_numpy(gt_box).cuda(),
                          vehi_reconstructor=vehi_reconstructor)
vehicles: list[vehicle_object] = scene.get_vehicles()
print(vehicles.__len__())

19


In [6]:
scene.pose_estimate(iter=25)

vis = o3d.visualization.Visualizer()
vis.create_window(window_name="view")
draw_scenes(vis, points=pts, gt_boxes=gt_box,
            draw_origin=True)

for vehi in vehicles:
    
    mesh = vehi.reconstruct_at_scene(vehi_reconstructor, standard_bbox)

    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(mesh.vertices),
        o3d.utility.Vector3iVector(mesh.faces))
    mesh = mesh.filter_smooth_simple(number_of_iterations=5)
    mesh.compute_vertex_normals()

    vis.add_geometry(mesh)
vis.run()
vis.destroy_window()

In [ ]:
for vehi in vehicles:
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="view")
    
    pts_centroid, bbox_centroid = vehi.centroid()
    point_colors = np.zeros((pts_centroid.shape[0], 3))
    point_colors[:, 0] = 1.0
    draw_scenes(vis, points=pts_centroid,
                gt_boxes=bbox_centroid.unsqueeze(dim=0), draw_origin=True, point_colors=point_colors)
    
    
    
    vehi.reconstruct_vehicle(vehi_reconstructor)
    vehicle_mesh: vehicle = vehi.vehicle
    vehicle_mesh = vehicle_mesh.to_trimesh()
    vehicle_mesh.vertices *= 0.1
    vehicle_mesh.vertices = vehicle_mesh.vertices[:, [2, 0, 1]]
    
    vehicle_mesh.vertices -= np.array([standard_bbox[2]/2.0,
                                      standard_bbox[0]/2.0,
                                      bbox_centroid[5].cpu() / 2.0])
    
    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(vehicle_mesh.vertices), 
        o3d.utility.Vector3iVector(vehicle_mesh.faces))
    # mesh = mesh.filter_smooth_simple(number_of_iterations=1)
    mesh.compute_vertex_normals()
    
    vis.add_geometry(mesh)
    vis.run()
    vis.destroy_window()

{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 69.118263244628906, 39.679920196533203, 16.415634155273438 ],
			"boundingbox_min" : [ -0.059999999999999998, -39.679874420166016, -6.9146575927734375 ],
			"field_of_view" : 60.0,
			"front" : [ -0.90307097537632919, 0.0017988087570628851, 0.42948757574567964 ],
			"lookat" : [ 34.529131622314452, 2.288818359375e-05, 4.75048828125 ],
			"up" : [ 0.42948904059539766, 0.0070563614983622357, 0.90304450154510629 ],
			"zoom" : 0.69999999999999996
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}